In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import seaborn as sns

In [2]:
# Inciar cartera(dict) y listas
cartera = {'link':'', 'nombre':'', 'precio':'', 'metros':'','tipologia':'', 'provincia':''}
links = []
nombres = []
precios = []
metros = []
tipologia = []
provincias = []


In [3]:
#funciones
def get_data(url):
    """Crea el beatiful soup con el link

    Args:
        url (str): url como string de la web

    Returns:
        BeautifulSoupObject: soup para trabajar sobre el html
    """
    url = requests.get(url).text
    soup = BeautifulSoup(url, 'lxml')
    return soup

def get_pages(n_page):
    """Devuelve el num de paginas totales

    Args:
        n_page (bs4.element): Soup a escrapear

    Returns:
        int: numero total de paginas
    """
    try:
        
        n_page = soup.find_all('div', class_ = 'haya-pagination mb-2')
        n_page = str(n_page)
        first_position = int(n_page.find('data-pagination-pages="'))   
        last_position = first_position + len('data-pagination-pages="')
        numero_paginas = n_page[last_position]
        
        for s in n_page[last_position + 1]:
            if s != '"':
                numero_paginas = numero_paginas + s
    except:
        numero_paginas = 1       
        
    return int(numero_paginas)
    #Sin uso habría que pasar el finder a trozos para que no sale el error
def error_handler(list,soup_finder):
    """This function does try except in order to create blank if error

    Args:
        list (list): list of asset attribute
        soup_finder (BeautifullSoup): Soup finder of the attribute
        
    returns attribute found or ''
    
    """
    try:
        return list.append(soup_finder)
    except:
        return list.append('')

In [3]:
#Sacar links viviendas / en caso de querer scrapear individualmente
url = requests.get('https://www.haya.es/comprar/viviendas/madrid/').text

soup = BeautifulSoup(url, 'lxml')

activos_com = soup.find_all('article', class_='col-12')

links = []

for activo in activos_com:

    try:
        links.append(activo.find('a', class_= 'text-black').get('href'))
        print(activo.find('a', class_= 'text-black').get('href'))
    except:
        links.append(activo.find('a', class_= 'text-white').get('href'))
        print(activo.find('a', class_= 'text-white').get('href'))
print(len(links))

https://www.haya.es/vivienda-7268726/
https://www.haya.es/vivienda-7042718/
https://www.haya.es/vivienda-7303940/
https://www.haya.es/vivienda-7233009/
https://www.haya.es/vivienda-6062604/
https://www.haya.es/vivienda-7236543/
https://www.haya.es/vivienda-7034916/
https://www.haya.es/vivienda-7297329/
https://www.haya.es/vivienda-5939450/
https://www.haya.es/vivienda-7293159/
https://www.haya.es/vivienda-7298192/
https://www.haya.es/vivienda-7293180/
https://www.haya.es/vivienda-7044980/
https://www.haya.es/vivienda-5957431/
https://www.haya.es/promocion-conjunta/promocion-1000020300/
https://www.haya.es/vivienda-7386653/
https://www.haya.es/vivienda-5932925/
https://www.haya.es/vivienda-7074792/
https://www.haya.es/vivienda-7268636/
https://www.haya.es/vivienda-182607/
https://www.haya.es/vivienda-5947188/
https://www.haya.es/vivienda-7039576/
https://www.haya.es/vivienda-5960379/
https://www.haya.es/vivienda-7043671/
https://www.haya.es/vivienda-7236157/
https://www.haya.es/promocio

In [4]:
provincias_list = ['A-Coruna','Alava','Albacete','Alicante','Almeria','Asturias','Avila','Badajoz','Baleares','Barcelona','Burgos','Caceres',\
    'Cadiz','Cantabria','Castellon','Ceuta','Cordoba','Cuenca','Gerona','Granada','Guadalajara','Guipuzcoa','Huelva','Jaen','La-Rioja','Las-Palmas',\
        'Leon','Lerida','Lugo','Madrid','Malaga','Melilla','Murcia','Navarra','Orense','Palencia','Pontevedra','Salamanca','Segovia','Sevilla','Soria',\
            'Tarragona','Tenerife','Teruel','Toledo','Valencia','Valladolid','Vizcaya','Zamora','Zaragoza'
]
tipologia_list = ['viviendas','terrenos','locales-comerciales','naves-industriales','edificios']

haya_url = ['https://www.haya.es/comprar/']

urls_iter =[]

for tip in tipologia_list:
    
    for prov in provincias_list:
        string = haya_url[0] + tip + '/' + prov + '/'
        urls_iter.append(string)


In [5]:
#Iterador final, itera sobre el frontpage para n paginas de cada url
paginas = urls_iter

for pag in paginas:
    
    nombre_provincia = re.findall('[A-Za-z]*\S\/',pag)[-1].replace('/','')
    
    nombre_tipologia = re.findall('[A-Za-z]*\S\/',pag)[3].replace('/','')
    
    url = pag
    
    soup = get_data(url)

    n_pages = get_pages(soup)

    activos = soup.find_all('article', class_='col-12')

    for i in range(n_pages):
                
        url_init = url
        
        if i != 0:
        
            url_str = url_init + '?p=' + str(i + 1)
            soup = get_data(url_str)
            activos = soup.find_all('article', class_='col-12')
            
        for activo in activos:

            if isinstance(activo.find('a', class_= 'text-black'),type(None)) == False:
                
                provincias.append(nombre_provincia)
                tipologia.append(nombre_tipologia)
                
                try:
                    links.append(activo.find('a', class_= 'text-black').get('href'))
                except:
                    links.append('')
                try:
                    nombres.append(activo.find('a', class_= 'text-black').get('title'))
                except:
                    nombres.append('')
                try:
                    precios.append(activo.find('span', class_='text-24 text-lg-28 text-bold ml-auto').text)
                except:
                    precios.append('')
                try:    
                    metros.append(activo.find('span', class_= 'text-20 text-medium').text) 
                except:
                    metros.append('')
            else:
                provincias.append(nombre_provincia)
                tipologia.append(nombre_tipologia)
                metros.append('')
                
                try:
                    links.append(activo.find('a', class_= 'text-white').get('href'))
                except:
                    links.append('')
                try:
                    nombres.append(activo.find('a', class_= 'text-white').text) 
                except:
                    nombres.append('')
                try:    
                    precios.append(activo.find('strong', class_='text-24 text-lg-28 text-bold').text)
                except:
                    precios.append('')
                
        
                    




C:\Users\Depoi\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:337: MarkupResemblesLocatorWarning: "..." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [7]:
#Actualizar cartera(dict) y crear cartera_haya como df

cartera['link'] = links
cartera['nombre'] = nombres
cartera['precio'] = precios
cartera['metros'] = metros
cartera['provincia'] = provincias
cartera['tipologia'] = tipologia

cartera_haya = pd.DataFrame(cartera)
cartera_haya['tipo precio'] = ''


In [9]:
#Limpiar Precios de strings

precio_limpio = cartera_haya['precio'].astype(str).replace('\t€','',regex=True).replace('\t','',regex=True)\
.replace(' €','',regex=True).replace(' ','',regex=True).replace('\.','',regex=True)

#Limpiar precios de consultar y desde y crear columna auxiliar
tipo_precio = []
precio_limpio_iter = pd.Series(precio_limpio, dtype='string')

for index, value in precio_limpio_iter.items():
    
   if 'desde' in precio_limpio_iter[index]:
      
      tipo_precio.append('Desde')
      precio_limpio_iter[index] = precio_limpio_iter[index].replace('desde','')
   
   if precio_limpio_iter[index] == 'consultarprecio' or precio_limpio_iter[index] == '0':
    
      tipo_precio.append('Consultar') 
      precio_limpio_iter[index] = '0'
      
   else:
      
      tipo_precio.append('Ok')
      
cartera_haya['precio'] = precio_limpio_iter.astype('int')
cartera_haya['tipo precio'] = tipo_precio      

#Limpiar Metros
metro_limpio =  cartera_haya['metros'].astype(str).replace('\t','',regex=True).replace('\.','',regex=True)

metro_limpio = metro_limpio.apply(lambda x: float(x.replace(',','.')) if x.replace(',','').isdigit() else 0)

metro_limpio = metro_limpio.apply(lambda x: round(x))

cartera_haya['metros'] = metro_limpio

#Obtener localidades
localidad_str=''
localidad_limpio = cartera_haya['nombre'].apply(lambda x: localidad_str.join(re.findall(r'(?<=\.).*?(?=\,)', x)))

#Añadir columna localidades
cartera_haya['localidad'] = localidad_limpio

In [75]:
metros_int = cartera_haya['metros']
# #.apply(lambda x: int(round(x)))

print(metros_int[8627])

print(cartera_haya.dtypes)


81
link         object
nombre       object
precio        int32
metros        int64
tipologia    object
provincia    object
localidad    object
dtype: object


In [10]:
#Crear excel
excel = cartera_haya
excel.to_excel(r'C:\Users\Depoi\OneDrive\Escritorio\Excel\Haya.xlsx', index = False)

In [121]:
#Dataframe final

filtro_provincias = cartera_haya.groupby('provincia')['tipologia'].value_counts()

print(filtro_provincias)

provincia  tipologia   
Alava      viviendas        10
           comerciales       7
           edificios         3
           terrenos          3
           industriales      1
                          ... 
Zaragoza   viviendas       200
           terrenos         67
           comerciales      26
           industriales     25
           edificios         1
Name: tipologia, Length: 209, dtype: int64


In [ ]:
sin_precio = cartera_haya['precio'] == 0
sin_localidad = cartera_haya[cartera_haya['localidad'] == '']
sin_metros = cartera_haya[cartera_haya['metros'] == 0]

print(len(sin_metros))
print(len(sin_precio[sin_precio == True]))
print(len(sin_localidad))

p_sin_precio = cartera_haya['precio'] == 0
p_sin_metros = cartera_haya['metros'] == 0
p_sin_localidad = cartera_haya['localidad'] == ''

print(p_sin_precio.mean(),p_sin_metros.mean(),p_sin_localidad.mean())

print(cartera_haya[cartera_haya['link'] == 'https://www.haya.es/vivienda-7045268/'].index.values)